# User based - Item based Recommendation System

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import surprise
from surprise import KNNBasic,accuracy
from surprise.model_selection import train_test_split

In [ ]:
ratings=pd.read_csv('../input/goodbooks-10k/ratings.csv')

In [ ]:
ratings.shape

In [ ]:
ratings.head()

In [ ]:
books=pd.read_csv('../input/goodbooks-10k/books.csv')

In [ ]:
books.shape

In [ ]:
books.head(20)

In [ ]:
train=pd.merge(books,ratings)
train.head()

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False)

In [ ]:
userRatings = train.pivot_table(index=['user_id'],columns=['title'],values='rating')
userRatings.head()

In [ ]:
userRatings=userRatings.dropna(thresh=10,axis=1).fillna(0)
#Here we are removing the movies having less than 10 user ratings and replacing NaN values with 0.
userRatings.head()

In [ ]:
books=userRatings.columns

In [ ]:
i=0
for book in books:
    print(i,book)
    i=i+1

In [ ]:
item_similarity_pearson=userRatings.corr(method='pearson')
item_similarity_pearson.head()

In [ ]:
ratings=ratings[['user_id','book_id','rating']] 
ratings = ratings.iloc[:20000,:]
reader = surprise.Reader(rating_scale=(1,5)) 
dataset1 = surprise.Dataset.load_from_df(ratings, reader)

In [ ]:
train1,test1 = train_test_split(dataset1,test_size=0.2)

## User Based

In [ ]:
user_model = surprise.KNNBasic(k=40,sim_options={'name': 'pearson','user_based': True})
user_model.fit(train1)
preds = user_model.test(test1)
accuracy.rmse(preds,verbose=True)

In [ ]:
user1=[("1421: The Year China Discovered America",4.0),
       ("A House for Mr Biswas",5.0),
       ("A History of God: The 4,000-Year Quest of Judaism, Christianity, and Islam",1.0)
      ]
user1=zip(books,userRatings.iloc[0,:])

In [ ]:
def get_similar_movies_pearson_user(movie_name,user_rating):
    similar_score=item_similarity_pearson[movie_name]*(user_rating-2.5)
    similar_score=similar_score.sort_values(ascending=False)
    return similar_score

In [ ]:
print("Using Pearson")
similar_movies=pd.DataFrame()
for movie,rating in user1:
    similar_movies=similar_movies.append(get_similar_movies_pearson_user(movie,rating),ignore_index=True)
recommended_pearson=similar_movies.sum().sort_values(ascending=False).index

In [ ]:
print("User-based Collabrative Filtering (Pearson)\n Recommendations for user_id 2")
print("\n\n")
for ind in recommended_pearson[:10]:
    print(ind)

## Item based

In [ ]:
item_model = surprise.KNNBasic(k=40,sim_options={'name': 'pearson','user_based': False})

In [ ]:
item_model.fit(train1)
preds = item_model.test(test1)
accuracy.rmse(preds,verbose=True)


In [ ]:
def get_similar_movies_pearson_itemtoitem(bookname):
    similar_score=item_similarity_pearson[bookname]*(2.5)
    similar_score=similar_score.sort_values(ascending=False)
    return similar_score

In [ ]:
book_name=books[0]

In [ ]:
recommended_pearson_item=get_similar_movies_pearson_itemtoitem(book_name).index

In [ ]:
print("Item based Collabrative Filtering for book",book_name,"\n\n")
for ind in recommended_pearson_item[1:11]:
    print(ind)

In [ ]:
nan